# Outlier Detection

This notebook will walk you through a workflow for outlier detection.

## TOC

 * [Detecting outlier jobs](#detect-outlier-jobs-study-1)
  * [partition jobs](#partition-jobs)
  * [create a trained model](#create-ref-model)
 * [Root Cause Analysis (RCA)](#rca-job)
 * [Detecting outlier operations](#detect-outlier-ops)
  * [RCA for operations](#rca-ops)
 * [Notes](#notes)

## <a name="detect-outlier-jobs-study-1">Detecting outlier jobs</a>

In this example we will use synthetic data. The data was generated using:
```
sample/outlier/workload.sh
```

The script basically compiles the linux kernel a few times. For certain compiles
it adds a background workload, so the compile takes longer. The slower job is marked
with the an 'outlier' suffix. However, we will pretend we don't know the outlier
and figure it out.

Along the way, we will also learn how to create a trained model, and use it 
for future outlier detection.


### Requirements

You will need to import data lying in `sample/outlier/*.tgz`:
```
$ ./epmt -v submit sample/outlier/*.tgz
```

In [1]:
import epmt_query as eq
import epmt_outliers as eod

INFO:epmt_job:Binding to DB: {'create_db': True, 'filename': 'database.sqlite', 'provider': 'sqlite'}
INFO:epmt_job:Generating mapping from schema...


In [2]:
jobs = eq.get_jobs(tag='exp_name:linux_kernel', fmt='terse')
jobs

['kern-6656-20190614-201744-outlier',
 'kern-6656-20190614-200819',
 'kern-6656-20190614-195909',
 'kern-6656-20190614-194953',
 'kern-6656-20190614-185359',
 'kern-6656-20190614-194024',
 'kern-6656-20190614-192044-outlier',
 'kern-6656-20190614-191138',
 'kern-6656-20190614-190245']

In [3]:
# As a first pass let's see whether the outliers can be auto-detected
(df, fdict) = eod.detect_outlier_jobs(jobs)
df

,jobid,duration,cpu_time,num_procs
0,kern-6656-20190614-185359,0,1,0
1,kern-6656-20190614-190245,0,0,0
2,kern-6656-20190614-191138,0,0,0
3,kern-6656-20190614-192044-outlier,1,1,0
4,kern-6656-20190614-194024,0,0,0
5,kern-6656-20190614-194953,0,0,0
6,kern-6656-20190614-195909,0,0,0
7,kern-6656-20190614-200819,0,0,0
8,kern-6656-20190614-201744-outlier,1,1,0


As you can see, while we did catch both the outliers, there
is also the "false positive" on one "non-outlier" process
The reason the 1 is marked for `duration` and `cpu_time` but `not num_procs`
is because the background compute process increased the job duration
but not the number of sub-processes of our workload.

### <a name="partition-jobs">Partitioning the jobs set into two sets</a>

`fdict`, obtained above, is a dictionary that contains for each feature 
(`cpu_time`, `duration` or `num_procs`), two sets, the first is a set of 
"reference" jobs and the second is a set of "outlier" jobs as detected by 
that particular feature.

In [4]:
fdict

{'cpu_time': ({'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819'},
  {'kern-6656-20190614-185359',
   'kern-6656-20190614-192044-outlier',
   'kern-6656-20190614-201744-outlier'}),
 'duration': ({'kern-6656-20190614-185359',
   'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819'},
  {'kern-6656-20190614-192044-outlier', 'kern-6656-20190614-201744-outlier'}),
 'num_procs': ({'kern-6656-20190614-185359',
   'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-192044-outlier',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819',
   'kern-6656-20190614-201744-outlier'},
  set())}

Partitioning can be more simply obtained using the following call:

In [5]:
parts = eod.partition_jobs(jobs, features=['duration'])
parts

{'duration': ({'kern-6656-20190614-185359',
   'kern-6656-20190614-190245',
   'kern-6656-20190614-191138',
   'kern-6656-20190614-194024',
   'kern-6656-20190614-194953',
   'kern-6656-20190614-195909',
   'kern-6656-20190614-200819'},
  {'kern-6656-20190614-192044-outlier', 'kern-6656-20190614-201744-outlier'})}

Above, we just got the partitioning of the jobs on a single `feature` -- `duration`.
You could also specify multiple features, such as `['duration', 'cpu_time']`.

### <a name="create-ref-model">Creating a trained (reference) model</a>

If we have a set of reference jobs, we can create a "trained" model. Subsequently we can do
outlier detection using the trained model. Let's use the reference set we obtained above:

In [6]:
ref_jobs = parts['duration'][0]
ref_jobs

{'kern-6656-20190614-185359',
 'kern-6656-20190614-190245',
 'kern-6656-20190614-191138',
 'kern-6656-20190614-194024',
 'kern-6656-20190614-194953',
 'kern-6656-20190614-195909',
 'kern-6656-20190614-200819'}

When creating the trained model, we can specify a `tag` of our choosing. This `tag` can
be used later to retrieve the trained model from the database.

In [7]:
r = eq.create_refmodel(ref_jobs, tag='exp_name:linux_kernel;type:ref')

In [8]:
r['id'], r['tags']

(4, {'exp_name': 'linux_kernel', 'type': 'ref'})

In [9]:
# using the trained model is as simple as:
(df, _) = eod.detect_outlier_jobs(jobs, trained_model = r['id'])
df

,jobid,duration,cpu_time,num_procs
0,kern-6656-20190614-185359,0,0,0
1,kern-6656-20190614-190245,0,0,0
2,kern-6656-20190614-191138,0,0,0
3,kern-6656-20190614-192044-outlier,1,1,0
4,kern-6656-20190614-194024,0,0,0
5,kern-6656-20190614-194953,0,0,0
6,kern-6656-20190614-195909,0,0,0
7,kern-6656-20190614-200819,0,0,0
8,kern-6656-20190614-201744-outlier,1,1,0


Obviously the jobs that were used to create the reference model will not be 
classifed as outliers for any feature.

## <a name="rca-job">Root Cause Analysis (RCA)</a>

In this study we will do an RCA with real data generated from GFDL PP runs.


### Requirements
Please import the following data:

```
$ ./epmt -v submit $(cat <<EOT
sample/ppr-batch/1854/625151.tgz
sample/ppr-batch/1859/627907.tgz
sample/ppr-batch/1869/633114.tgz
sample/ppr-batch/1864/629322.tgz
sample/ppr-batch/1884/685001.tgz
sample/ppr-batch/1874/675992.tgz
sample/ppr-batch/1879/680163.tgz
sample/ppr-batch/1889/691209.tgz
sample/ppr-batch/1894/693129.tgz
EOT
)
```

All these jobs share the following tags: `{u'ocn_res': u'0.5l75', u'atm_res': u'c96l49', u'exp_component': u'ocean_annual_z_1x1deg', u'exp_name': u'ESM4_historical_D151'}`. The difference is only that they have different values for `('exp_time', 'script_name')`.
<div></div>

<!--
<details>
  <summary>Advanced query</summary>
  If you are curious how we found these comparable jobs, here is the query (ADVANCED TOPIC):
  ```
>>> x = eq.Job.select(lambda j: j.tags['exp_component'] == 'ocean_annual_z_1x1deg').filter(lambda j: j.tags['exp_name'] == 'ESM4_historical_D151')
>>> eq.get_jobs(x, fmt="terse")
  ```
</details>
-->

In [10]:
jobs = eq.get_jobs(tag="exp_name:ESM4_historical_D151;exp_component:ocean_annual_z_1x1deg", fmt='terse')
jobs

['693129',
 '691209',
 '680163',
 '675992',
 '685001',
 '629322',
 '633114',
 '627907',
 '625151']

Now, let's partition the jobs by `cpu_time`: 

In [11]:
parts = eod.partition_jobs(jobs, features=['cpu_time'])
parts

{'cpu_time': ({'627907',
   '629322',
   '633114',
   '675992',
   '680163',
   '685001',
   '691209',
   '693129'},
  {'625151'})}

As you can see, the first partition contains conformant jobs, and the outlier
partition contains a single job `625151`. Let's see how different is the `cpu_time`
of this job v. the rest:

In [12]:
jobs_df = eq.get_jobs(jobs, fmt='pandas', order='desc(j.cpu_time)')
display(jobs_df.columns.values)
jobs_df[['jobid', 'cpu_time', 'duration', 'num_procs']]

array(['PERF_COUNT_SW_CPU_CLOCK', 'account', 'all_proc_tags',
       'cancelled_write_bytes', 'cpu_time', 'created_at',
       'delayacct_blkio_time', 'duration', 'end', 'env_changes_dict',
       'env_dict', 'exitcode', 'guest_time', 'inblock', 'info_dict',
       'invol_ctxsw', 'jobid', 'jobname', 'jobscriptname', 'majflt',
       'minflt', 'num_procs', 'num_threads', 'outblock', 'processor',
       'queue', 'rchar', 'rdtsc_duration', 'read_bytes', 'rssmax',
       'sessionid', 'start', 'submit', 'syscr', 'syscw', 'systemtime',
       'tags', 'time_oncpu', 'time_waiting', 'timeslices', 'updated_at',
       'user', 'user+system', 'usertime', 'vol_ctxsw', 'wchar',
       'write_bytes'], dtype=object)

,jobid,cpu_time,duration,num_procs
0,625151,1224444629,10425623185,13530
1,627907,694594906,6589174875,4411
2,629322,622137956,7286331754,4411
3,691209,536679993,860163243,4411
4,693129,533686454,3619324767,4411
5,675992,520837207,9114150525,4411
6,685001,480310481,6815710476,4428
7,680163,462936089,6156192011,4411
8,633114,457180929,6036720046,4445


It's really interesting that the outlier has thrice the number of processes
as the others. Let's see if we can uncover more..

In [13]:
(refs, outl) = parts['cpu_time']
(refs, outl)

({'627907',
  '629322',
  '633114',
  '675992',
  '680163',
  '685001',
  '691209',
  '693129'},
 {'625151'})

In [14]:
(_, df, flist) = eod.detect_rootcause(refs, '625151')
df

,num_procs,cpu_time,duration
count,8.000000,8.000000e+00,8.000000e+00
mean,4417.375000,5.385455e+08,5.809721e+09
std,12.648405,8.229571e+07,2.512447e+09
min,4411.000000,4.571809e+08,8.601632e+08
25%,4411.000000,4.759669e+08,5.432371e+09
50%,4411.000000,5.272618e+08,6.372683e+09
75%,4415.250000,5.580445e+08,6.933366e+09
max,4445.000000,6.945949e+08,9.114151e+09
input,13530.000000,1.224445e+09,1.042562e+10
ref_max_modified_z_score,3.597300,2.028600e+00,5.481300e+00


The features are ranked from the one with the highest `modified_z_score_ratio` from left to right, 
in decreasing importance.

In [15]:
flist

[('num_procs', 268.2083785061018),
 ('cpu_time', 4.166370896184561),
 ('duration', 0.735227044679182)]

If you would like to expand the RCA to other `features`, do as below:

In [16]:
(_, df, flist) = eod.detect_rootcause(refs, '625151', features=[])
df

,wchar,syscw,rchar,num_threads,num_procs,rssmax,syscr,vol_ctxsw,timeslices,minflt,...,invol_ctxsw,read_bytes,inblock,cancelled_write_bytes,majflt,rdtsc_duration,exitcode,delayacct_blkio_time,guest_time,processor
count,8.000000e+00,8.000000,8.000000e+00,8.000000,8.000000,8.000000e+00,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000e+00,8.000000,8.000000e+00,8.000000,8.000000e+00,8,8,8,8
mean,7.272861e+10,4052585.500000,9.568890e+10,4682.750000,4417.375000,6.962899e+07,12201808.000000,823488.875000,850609.750000,7527056.375000,...,22424.625000,9.678984e+09,18904257.000000,2.056820e+09,558.375000,-1.012455e+16,0,0,0,0
std,8.307445e+04,317.118725,2.773749e+06,13.392429,12.648405,2.277324e+05,5538.602738,20708.215233,28892.369866,212811.545768,...,21975.570631,1.439832e+10,28121705.822046,5.042457e+08,1298.907226,2.000890e+16,0,0,0,0
min,7.272851e+10,4052375.000000,9.568720e+10,4676.000000,4411.000000,6.927494e+07,12198338.000000,802024.000000,820315.000000,7328295.000000,...,12130.000000,3.448832e+06,6736.000000,8.088945e+08,2.000000,-5.328561e+16,0,0,0,0
25%,7.272854e+10,4052453.000000,9.568735e+10,4676.000000,4411.000000,6.957019e+07,12198807.750000,811829.750000,829806.250000,7372435.000000,...,13439.750000,1.447434e+08,282702.000000,2.234230e+09,11.750000,-6.987199e+15,0,0,0,0
50%,7.272860e+10,4052482.500000,9.568751e+10,4676.000000,4411.000000,6.962996e+07,12199156.000000,813721.000000,838391.500000,7527901.500000,...,14215.500000,1.980019e+09,3867224.000000,2.236223e+09,45.000000,3.984634e+13,0,0,0,0
75%,7.272867e+10,4052528.750000,9.568891e+10,4680.500000,4415.250000,6.965483e+07,12202047.250000,832198.250000,871352.000000,7543403.250000,...,16518.500000,1.461061e+10,28536334.000000,2.236244e+09,220.500000,5.426578e+13,0,0,0,0
max,7.272873e+10,4053353.000000,9.569491e+10,4712.000000,4445.000000,7.007920e+07,12214610.000000,862453.000000,895097.000000,8003186.000000,...,76421.000000,4.000315e+10,78131160.000000,2.236248e+09,3750.000000,7.181059e+13,0,0,0,0
input,1.388007e+11,4436801.000000,9.854065e+10,14551.000000,13530.000000,1.336608e+08,13084676.000000,3150991.000000,3198404.000000,20550722.000000,...,32844.000000,1.281107e+10,25021616.000000,2.331615e+09,80.000000,1.009561e+14,0,0,0,0
ref_max_modified_z_score,1.223000e+00,13.979800,2.127270e+01,3.597300,3.597300,5.644000e+00,17.727400,4.232200,2.765000,3.701900,...,38.777800,1.298700e+01,12.987000,4.700847e+04,62.475600,1.580507e+03,0,0,0,0


Note that the metrics are aggregates across the underlying threads/processes across a job.

In our next study we will attempt to determine which `operations` from within the jobs were outliers.

## <a name="detect-outlier-ops">Detect Outlier Operations</a>

This study shows how we can find outlier operations across a set of jobs.

Please [review the previous study](#rca-job) as this has the same requirements.

In [17]:
jobs = eq.get_jobs(tag="exp_name:ESM4_historical_D151;exp_component:ocean_annual_z_1x1deg", fmt='terse')

In [18]:
len(jobs)

9

In [19]:
# widen width of column display width to show full tag
import pandas as pd
pd.set_option('display.max_colwidth', 200)
(df, parts, scores_df, sorted_tags, sorted_features) = eod.detect_outlier_ops(jobs)
df.head(20)

,jobid,tags,duration,cpu_time,num_procs
0,625151,"{'op': 'hsmget', 'op_sequence': '21', 'op_instance': '6'}",0,0,0
1,627907,"{'op': 'hsmget', 'op_sequence': '21', 'op_instance': '6'}",0,0,0
2,629322,"{'op': 'hsmget', 'op_sequence': '21', 'op_instance': '6'}",1,0,0
3,633114,"{'op': 'hsmget', 'op_sequence': '21', 'op_instance': '6'}",1,0,0
4,675992,"{'op': 'hsmget', 'op_sequence': '21', 'op_instance': '6'}",1,0,0
5,680163,"{'op': 'hsmget', 'op_sequence': '21', 'op_instance': '6'}",0,0,0
6,685001,"{'op': 'hsmget', 'op_sequence': '21', 'op_instance': '6'}",0,1,0
7,691209,"{'op': 'hsmget', 'op_sequence': '21', 'op_instance': '6'}",0,0,0
8,693129,"{'op': 'hsmget', 'op_sequence': '21', 'op_instance': '6'}",0,0,0
9,625151,"{'op': 'hsmget', 'op_sequence': '19', 'op_instance': '7'}",0,0,0


In [20]:
len(sorted_tags)

416

In [21]:
sorted_tags[:5]

[{'op': 'hsmget', 'op_instance': '6', 'op_sequence': '21'},
 {'op': 'hsmget', 'op_instance': '7', 'op_sequence': '19'},
 {'op': 'hsmget', 'op_instance': '6', 'op_sequence': '18'},
 {'op': 'hsmget', 'op_instance': '7', 'op_sequence': '13'},
 {'op': 'mv', 'op_instance': '10', 'op_sequence': '60'}]

As you can see there are 416 unique operations. Since we want to focus our attention
on the operations, that have the highest deviation across jobs, `detect_outlier_ops`
helps us by ordering the dataframe, `df` in the order of decreasing operation (tag)
importance. To figure out the importance of a tag, it uses the maximum of the scores
across all jobs, across all features for that particular tag. `scores_df` and `sorted_tags`
are similarly ordered by decreasing tag importance.

In [22]:
scores_df.head()

,tags,duration,cpu_time,num_procs
0,"{""op"": ""hsmget"", ""op_instance"": ""6"", ""op_sequence"": ""21""}",646.428,0.026,0
1,"{""op"": ""hsmget"", ""op_instance"": ""7"", ""op_sequence"": ""19""}",463.380,3.166,0
2,"{""op"": ""hsmget"", ""op_instance"": ""6"", ""op_sequence"": ""18""}",362.595,0.000,0
3,"{""op"": ""hsmget"", ""op_instance"": ""7"", ""op_sequence"": ""13""}",229.944,0.000,0
4,"{""op"": ""mv"", ""op_instance"": ""10"", ""op_sequence"": ""60""}",160.865,0.000,0


`detect_outlier_ops` thus already helps with RCA by ordering the output
in decreasing tag importance. It goes even further, by presenting a ordered
`features` list, in descreasing order of feature importance. Here the
importance of a `feature` is determined by summing the scores of the
feature across all tags. 

In [23]:
sorted_features

['duration', 'cpu_time', 'num_procs']

<a name="rca-ops"></a>Once you have the ordered list of tags (operations), you may want to do
further RCA analysis for a specific operation. For RCA analysis at the operation-level, 
we specify a *reference set* of jobs and an outlier job (similar to how we did for RCA at the job-level). 
In addition, we specify the operation of interest. The goal of the op-RCA is to rank the 
features in order of importance.

Let's suppose we care about the top operation: `{"op": "ncatted", "op_instance": "3", "op_sequence": "32"}`.
We see the jobid `629322` was one of the outliers for this operation in `df` for both the `duration` and
`cpu_time` features.

In [24]:
# first derive the list of jobs other than the outlier
refjobs = eq.get_jobs(tag="exp_name:ESM4_historical_D151;exp_component:ocean_annual_z_1x1deg", fltr='j.jobid != "629322"', fmt='terse')
refjobs

['693129',
 '691209',
 '680163',
 '675992',
 '685001',
 '633114',
 '627907',
 '625151']

In [25]:
(ret, df_rca, feature_scores) = eod.detect_rootcause_op(refjobs, '629322', {"op": "ncatted", "op_instance": "3", "op_sequence": "32"})
ret

True

In [26]:
df_rca

,duration,cpu_time,num_procs
count,7.000000,7.000000,7
mean,1188.428571,24566.571429,1
std,114.402589,4197.045503,0
min,1109.000000,21995.000000,1
25%,1120.000000,22995.000000,1
50%,1149.000000,22995.000000,1
75%,1196.500000,23496.000000,1
max,1428.000000,33994.000000,1
input,1735.000000,29994.000000,1
ref_max_modified_z_score,4.952200,7418.825500,0


In [27]:
# to get the full feature list, just pass features=[] to the same function
(ret, df_rca, feature_scores) = eod.detect_rootcause_op(refjobs, '629322', {"op": "ncatted", "op_instance": "3", "op_sequence": "32"}, features=[])
ret

True

In [28]:
df_rca

,read_bytes,majflt,inblock,timeslices,vol_ctxsw,rdtsc_duration,minflt,duration,systemtime,cpu_time,...,syscr,delayacct_blkio_time,write_bytes,syscw,numtids,outblock,guest_time,cancelled_write_bytes,processor,rssmax
count,7.000000e+00,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,...,7.000000,7,7,7,7,7,7,7,7,7.000000
mean,0.000000e+00,0.000000,0.000000,22.142857,17.714286,4033804.142857,2524.857143,1188.428571,7141.142857,24566.571429,...,281.571429,0,4096,3,1,8,0,0,0,7984.571429
std,0.000000e+00,0.000000,0.000000,1.345185,1.889822,273511.549812,27.431299,114.402589,1463.476048,4197.045503,...,1.511858,0,0,0,0,0,0,0,0,36.836835
min,0.000000e+00,0.000000,0.000000,21.000000,17.000000,3813848.000000,2508.000000,1109.000000,4999.000000,21995.000000,...,281.000000,0,4096,3,1,8,0,0,0,7968.000000
25%,0.000000e+00,0.000000,0.000000,21.500000,17.000000,3854039.000000,2509.000000,1120.000000,6498.500000,22995.000000,...,281.000000,0,4096,3,1,8,0,0,0,7970.000000
50%,0.000000e+00,0.000000,0.000000,22.000000,17.000000,3938599.000000,2509.000000,1149.000000,6998.000000,22995.000000,...,281.000000,0,4096,3,1,8,0,0,0,7972.000000
75%,0.000000e+00,0.000000,0.000000,22.000000,17.000000,4113005.000000,2533.500000,1196.500000,7998.000000,23496.000000,...,281.000000,0,4096,3,1,8,0,0,0,7972.000000
max,0.000000e+00,0.000000,0.000000,25.000000,22.000000,4550094.000000,2572.000000,1428.000000,8998.000000,33994.000000,...,285.000000,0,4096,3,1,8,0,0,0,8068.000000
input,4.915200e+04,1.000000,96.000000,140.000000,136.000000,5978859.000000,2644.000000,1735.000000,9998.000000,29994.000000,...,281.000000,0,4096,3,1,8,0,0,0,7972.000000
ref_max_modified_z_score,0.000000e+00,0.000000,0.000000,2.832900,4.721500,3.583700,42.493500,4.952200,1.350400,7418.825500,...,4.721500,0,0,0,0,0,0,0,0,4.358300


## <a name="notes">Notes and Suggestions</a>

 * When we do `detect_outlier_ops` we might also want to know the top jobs (ranked) by tag.
   This would help for subsequent RCA analysis.
 * The columns of `df` and `scores_df` should be ordered in the same order as `sorted_features`.
